In [ ]:
library(rtweet)        # Used for extracting the tweets
library(tm)            # Text mining cleaning
library(stringr)       # Removing characters
library(qdapRegex)     # Removing URLs 
library(wordcloud2)    # Creating the wordcloud
library(httpuv)
library(plyr)
library(NLP)
library(syuzhet)
library(ggplot2)

In [ ]:
## store api keys (these are empty example values; replace with your own keys)
api_key <- "xxxxxxxxxxxx"
api_secret_key <- "xxxxxxxxxxxxxxxxxxxxx"

## authenticate via web browser
token <- create_token(
  app = "random1258475983",
  consumer_key = api_key,
  consumer_secret = api_secret_key)

In [ ]:
## most recent to least tweet order. Can set to 18000 I believe is the limit.
tweets <- get_timelines(c("twitter_account"), n = 18000)

In [ ]:
twitter <- search_tweets(q = "words or phrase to search for in twitter", n = 18000, lang = "en")

In [ ]:
## simple text cleaning
text <- str_c(tweets$text, collapse = "")

In [ ]:
## simple text cleaning
text2 <- str_c(twitter$text, collapse = "")

In [ ]:
## more text cleaning 
text <- 
  text %>%
  str_remove("\\n") %>%                   # remove linebreaks
  rm_twitter_url() %>%                    # Remove URLS
  rm_url() %>%
  str_remove_all("#\\S+") %>%             # Remove any hashtags
  str_remove_all("@\\S+") %>%             # Remove any @ mentions
  removeWords(stopwords("english")) %>%   # Remove common words (a, the, it etc.)
  removeNumbers() %>%
  stripWhitespace() %>%
  removeWords(c("amp"))                   # Final cleanup of other small things

In [ ]:
## more text cleaning 
text2 <- 
  text %>%
  str_remove("\\n") %>%                   # remove linebreaks
  rm_twitter_url() %>%                    # Remove URLS
  rm_url() %>%
  str_remove_all("#\\S+") %>%             # Remove any hashtags
  str_remove_all("@\\S+") %>%             # Remove any @ mentions
  removeWords(stopwords("english")) %>%   # Remove common words (a, the, it etc.)
  removeNumbers() %>%
  stripWhitespace() %>%
  removeWords(c("amp"))                   # Final cleanup of other small things

In [ ]:
# Convert the data into a summary table
textCorpus <- 
  Corpus(VectorSource(text)) %>%
  TermDocumentMatrix() %>%
  as.matrix()

textCorpus <- sort(rowSums(textCorpus), decreasing=TRUE)
textCorpus <- data.frame(word = names(textCorpus), freq=textCorpus, row.names = NULL)

In [ ]:
# Convert the data into a summary table
textCorpus <- 
  Corpus(VectorSource(text2)) %>%
  TermDocumentMatrix() %>%
  as.matrix()

textCorpus <- sort(rowSums(textCorpus), decreasing=TRUE)
textCorpus <- data.frame(word = names(textCorpus), freq=textCorpus, row.names = NULL)

In [ ]:
# build wordcloud 
wordcloud <- wordcloud(data = textCorpus, minRotation = 0, maxRotation = 0, ellipticity = 0.6)

In [ ]:
wordcloud

In [ ]:
# build wordcloud 
wordcloud2 <- wordcloud2(data = textCorpus, minRotation = 0, maxRotation = 0, ellipticity = 0.6)

In [ ]:
wordcloud2

In [ ]:
## can group a bunch of twitter accounts together 

TweetsToWordcloud <- function(username){
  
  tweets <- get_timelines(username, n = 18000)
  
  # Clean the data
  text <- str_c(tweets$text, collapse = "") %>%
  str_remove("\\n") %>%                   # remove linebreaks
  rm_twitter_url() %>%                    # Remove URLS
  rm_url() %>%
  str_remove_all("#\\S+") %>%             # Remove any hashtags
  str_remove_all("@\\S+") %>%             # Remove any @ mentions
  removeWords(stopwords("english")) %>%   # Remove common words (a, the, it etc.)
  removeNumbers() %>%
  stripWhitespace() %>%
  removeWords(c("amp"))                   # Final cleanup of other small changes
  
    # Convert the data into a summary table
  textCorpus <- 
    Corpus(VectorSource(text)) %>%
    TermDocumentMatrix() %>%
    as.matrix()
  
  textCorpus <- sort(rowSums(textCorpus), decreasing=TRUE)
  textCorpus <- data.frame(word = names(textCorpus), freq=textCorpus, row.names = NULL)
  
  wordcloud <- wordcloud2(data = textCorpus, minRotation = 0, maxRotation = 0, ellipticity = 0.6)
  return(wordcloud)
}

In [ ]:
TweetsToWordcloud(username = "twitter_account")

In [ ]:
##Continuing process to get sentiment analysis of twitter phrases or accounts
sentiment<-get_nrc_sentiment((text))

In [ ]:
Sentimentscores<-data.frame(colSums(sentiment[,]))

In [ ]:
names(Sentimentscores)<-"Score"
Sentimentscores<-cbind("sentiment"=rownames(Sentimentscores),Sentimentscores)
rownames(Sentimentscores)<-NULL

In [ ]:
ggplot(data=Sentimentscores,aes(x=sentiment,y=Score))+geom_bar(aes(fill=sentiment),stat = "identity")+
  theme(legend.position="none")+
  xlab("Sentiments")+ylab("Scores")+ggtitle("")